# Segmentation Model Benchmarking & Comparative Analysis

This notebook provides a professional workflow for benchmarking and comparing segmentation models (DeepLabV3+, GhanaSegNet, SegFormer, UNet) on Ghanaian food image datasets. It includes file upload, metric visualization, and thesis-ready summary.

## Upload Your Results Files
Before running the analysis, use the cell below to upload your JSON result files (`deeplabv3plus_results.json`, `ghanasegnet_results.json`, `segformer_results.json`, `unet_results.json`).
Click the "Choose Files" button when you run the upload cell and select all four files.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EricBaidoo/GhanaSegNet/blob/main/analysis/Segmentation_Model_Benchmarking_Analysis.ipynb)

---

In [ ]:
# Install required libraries
!pip install matplotlib

In [ ]:
# Upload results files (Colab only)
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load results
import json
import matplotlib.pyplot as plt

FILES = [
    'deeplabv3plus_results.json',
    'ghanasegnet_results.json',
    'segformer_results.json',
    'unet_results.json'
 ]
models = []
for fname in FILES:
    with open(fname, 'r') as f:
        models.append(json.load(f))

In [ ]:
# Plot training metrics
metrics = ['val_iou', 'val_loss', 'val_accuracy']
for metric in metrics:
    plt.figure(figsize=(8, 5))
    for m in models:
        epochs = [h['epoch'] for h in m['training_history']]
        values = [h[metric] for h in m['training_history']]
        plt.plot(epochs, values, label=m['model_name'])
    plt.xlabel('Epoch')
    plt.ylabel(metric.replace('_', ' ').title())
    plt.title(f'{metric.replace('_', ' ').title()} Over Epochs')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Model comparison summary
for m in models:
    print(f"Model: {m['model_name']}")
    print(f"  Best IoU: {m['best_iou']:.4f}")
    print(f"  Final Epoch: {m['final_epoch']}")
    print(f"  Parameters: {m['trainable_parameters']:,}")
    print()

### Thesis-Ready Comparative Summary
The comparative analysis of four segmentation models—DeepLabV3+, GhanaSegNet, SegFormer, and UNet—reveals distinct performance characteristics. DeepLabV3+ achieved the highest IoU (0.2544), indicating superior segmentation accuracy, but it is also the most parameter-heavy. GhanaSegNet, while slightly trailing in IoU (0.2447), is significantly more parameter-efficient, making it attractive for resource-constrained applications. SegFormer and UNet plateaued early, suggesting limited learning capacity or data/model constraints under current settings. All models were trained under identical conditions for fairness. The results highlight GhanaSegNet’s competitive performance and efficiency, with recommendations to further improve its accuracy through advanced augmentation, loss functions, or architectural refinements. Future work may explore deeper architectures, alternative backbones, or larger datasets to enhance segmentation outcomes.